# KicadAutoFlow Inventory Management
This notebook helps you manage your physical component inventory for KiCad projects:
- View and explore your existing inventory
- Easily identify components from images using AI
- Verify and add components to inventory
- Search and filter your inventory

In [1]:
# Setup imports and configurations
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Image, clear_output
from pathlib import Path
from utils.config_loader import load_config
from adk_capabilities.llm_capability import GeminiCapability
from inventory_manager import Inventory, ImageAnalysisManager

# Load configuration
config = load_config('config.yaml')

# Initialize LLM capability
llm = GeminiCapability(config=config)

# Initialize inventory
inv_path = Path('inventory.yaml')
inventory = Inventory(inv_path)
inventory.load()

# Initialize image analysis manager
img_dir = Path('docs/inventory_images')
img_dir.mkdir(parents=True, exist_ok=True)
image_analyzer = ImageAnalysisManager(llm, inventory, img_dir)

Gemini configured successfully.
GeminiCapability Initialized (API Active: True)
Loaded 0 items from 'C:\Temp\KicadAutoFlow\inventory.yaml'.


## 1. View Inventory
Display your current inventory of components.

In [2]:
def display_inventory():
    # Reload inventory to get latest updates
    inventory.load()
    df = inventory.to_dataframe()
    if len(df) > 0:
        display(df)
        print(f"\nTotal components: {len(df)}")
    else:
        print("Inventory is empty. Use the tools below to add components.")
    return df

display_inventory()

Loaded 0 items from 'C:\Temp\KicadAutoFlow\inventory.yaml'.
Inventory is empty. Use the tools below to add components.


,part_id,description,value,package,footprint,footprint_source,mpn,quantity,storage_location,datasheet_local,image_path,mounting_type,analysis_confidence


## 2. Image-Based Component Identification

In [ ]:
# default image path - downloads folder
raw_image_path = os.path.join(os.path.expanduser("~"), "Downloads")
images = ["part.jpg"]

image_analyzer = ImageAnalysisManager(llm, inventory, img_dir)
res_components = []
for image in images:
    image_path = os.path.join(raw_image_path, image)
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        continue
    
    print(f"Analyzing image: {image}")
    
    # Use the ImageAnalysisManager to analyze the image
    item = image_analyzer.analyze_image(image_path)
    if item is None:
        print("❌ Could not identify a component from the image.")
        continue

    res_components += [item]
    print("\nIdentified Component:")
    print(item)

Analyzing image: part.jpg
Image analysis result: {'detected_text': 'HYSD Store, 50pcs RJ11 Telephone Jack 95001, 95001-4P4C, 95001-4p4c, 95001-6p4c, 95001-6p4c, 95001-6p4c', 'component_type_guess': 'Connector', 'package_guess': None, 'value_guess': None, 'mpn': '95001-4P4C', 'quantity_guess': 50, 'mounting_type': 'Through-Hole', 'pin_count_guess': 4, 'color': 'Black', 'markings_position': 'Top', 'damages': None, 'confidence': 'High', 'is_kit': True, 'kit_description': 'A kit of 50 RJ11 telephone jacks.', 'component_types': ['Connector'], 'approximate_total_count': 50}
Error analyzing image: 1 validation error for InventoryItem
footprint_source
  Input should be 'manual', 'api_verified', 'kit_ingest_verified' or 'unknown' [type=literal_error, input_value='image_analysis', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/literal_error

Identified Component:
Image analysis result: {'detected_text': 'HYSD Store, 50pcs RJ11 Telephone Jack 95001, 95001-4P4

AttributeError: 'dict' object has no attribute 'pretty_print'

## 3. Add to Inventory
Verify the component and add it to your inventory.

In [ ]:
# Create verification form widgets
edit_desc = widgets.Text(description='Description:', placeholder='e.g., Resistor, Capacitor')
edit_value = widgets.Text(description='Value:', placeholder='e.g., 10K, 100nF')
edit_package = widgets.Text(description='Package:', placeholder='e.g., 0805, DIP-8')
edit_footprint = widgets.Text(description='Footprint:', placeholder='e.g., Resistor_SMD:R_0805_2012Metric')
edit_mounting = widgets.Dropdown(
    description='Mounting:',
    options=['Surface Mount', 'Through-Hole', 'Mixed', 'Unknown'],
    value='Unknown'
)
edit_quantity = widgets.IntText(description='Quantity:', value=1, min=1)
edit_mpn = widgets.Text(description='MPN:', placeholder='Manufacturer Part Number (optional)')
edit_storage = widgets.Text(description='Storage:', placeholder='Storage location (optional)')

# Create add to inventory button
add_btn = widgets.Button(description="Add to Inventory", button_style="success")

def on_add_clicked(b):
    if not last_analyzed_component:
        print("No component to add. Please analyze an image first.")
        return
        
    # Update component with edited values
    if edit_desc.value: last_analyzed_component.description = edit_desc.value
    if edit_value.value: last_analyzed_component.value = edit_value.value
    if edit_package.value: last_analyzed_component.package = edit_package.value
    if edit_footprint.value: last_analyzed_component.footprint = edit_footprint.value
    last_analyzed_component.mounting_type = edit_mounting.value
    last_analyzed_component.quantity = edit_quantity.value
    if edit_mpn.value: last_analyzed_component.mpn = edit_mpn.value
    if edit_storage.value: last_analyzed_component.storage_location = edit_storage.value
    
    # Add to inventory
    inventory.add_part(last_analyzed_component)
    inventory.save()
    
    # Clear form
    clear_output(wait=True)
    display(widgets.HTML('<h3>Upload a component image</h3>'))
    display(upload)
    display(analyze_btn)
    
    print(f"✅ Added {last_analyzed_component.part_id} to inventory.")
    
    # Refresh inventory display
    display_inventory()
    
add_btn.on_click(on_add_clicked)

def display_edit_form():
    if not last_analyzed_component:
        return
        
    # Prefill form with component values
    edit_desc.value = last_analyzed_component.description
    edit_value.value = last_analyzed_component.value or ''
    edit_package.value = last_analyzed_component.package or ''
    edit_footprint.value = last_analyzed_component.footprint or ''
    edit_mounting.value = last_analyzed_component.mounting_type or 'Unknown'
    edit_quantity.value = last_analyzed_component.quantity
    edit_mpn.value = last_analyzed_component.mpn or ''
    edit_storage.value = last_analyzed_component.storage_location or ''
    
    # Display form
    form = widgets.VBox([
        widgets.HTML('<h3>Verify Component Information</h3>'),
        edit_desc, edit_value, edit_package, edit_footprint,
        edit_mounting, edit_quantity, edit_mpn, edit_storage
    ])
    display(form)

## 4. Analyze Existing Image
Analyze an image that already exists on your system.

In [ ]:
def analyze_from_path(image_path):
    """Analyze a component from an existing image path."""
    # Convert to Path object if it's a string
    path = Path(image_path)
    
    if not path.exists():
        print(f"Error: Image not found at {path}")
        return None
        
    # Display the image
    with open(path, 'rb') as f:
        img_data = f.read()
        display(Image(data=img_data))
    
    # Analyze the image
    print(f"Analyzing image: {path.name}")
    component = image_analyzer.add_part_from_image(path, save_to_inventory=False)
    
    print("\nIdentified Component:")
    print(component.pretty_print())
    
    return component

# Example usage:
# component = analyze_from_path('docs/inventory_images/resistor.jpg')
# if component:
#     inventory.add_part(component)
#     inventory.save()
#     print(f"Added {component.part_id} to inventory")

## 5. Bulk Image Analysis
Analyze multiple component images from a directory.

In [ ]:
def analyze_directory(directory_path):
    """Analyze all images in a directory."""
    # Convert to Path object if it's a string
    path = Path(directory_path)
    
    if not path.exists() or not path.is_dir():
        print(f"Error: Directory not found at {path}")
        return
        
    # Get all image files
    image_files = list(path.glob('*.jpg')) + list(path.glob('*.jpeg')) + \
                  list(path.glob('*.png')) + list(path.glob('*.gif'))
    
    if not image_files:
        print(f"No image files found in {path}")
        return
        
    print(f"Found {len(image_files)} image files. Processing...")
    
    # Process each image
    results = []
    for img_path in image_files:
        print(f"\nProcessing {img_path.name}...")
        component = image_analyzer.add_part_from_image(img_path, save_to_inventory=False)
        results.append(component)
        print(f"Identified as: {component.description} {component.value}")
    
    return results

# Example usage:
# components = analyze_directory('docs/inventory_images/kit1')
# if components:
#     for component in components:
#         inventory.add_part(component)
#     inventory.save()
#     print(f"Added {len(components)} components to inventory")

## 6. Search Inventory
Search for components in your inventory.

In [ ]:
# Create bulk import widgets
bulk_upload = widgets.FileUpload(
    accept='image/*', 
    multiple=False,
    description='Upload Kit Image:'
)
bulk_analyze_btn = widgets.Button(description='Analyze Kit', button_style='info', disabled=True)
bulk_output = widgets.Output()

# Display bulk import section
display(widgets.VBox([
    widgets.HTML('<h3>Bulk Import from Component Kit</h3>'),
    widgets.HTML('<p>Upload an image of a component kit list to analyze multiple components</p>'),
    widgets.HBox([bulk_upload, bulk_analyze_btn]),
    bulk_output
]))

# Global variables for bulk import
bulk_img_path = None
bulk_img_bytes = None

# Handle bulk image upload
def on_bulk_upload_change(change):
    global bulk_img_path, bulk_img_bytes
    
    with bulk_output:
        clear_output()
        
        if bulk_upload.value:
            img_info = list(bulk_upload.value.values())[0]
            bulk_img_bytes = img_info['content']
            img_name = f"kit_{img_info['name']}"
            
            bulk_img_path = img_dir / img_name
            with open(bulk_img_path, 'wb') as f:
                f.write(bulk_img_bytes)
                
            display(Image(data=bulk_img_bytes))
            print(f'Kit image saved to {bulk_img_path}')
            bulk_analyze_btn.disabled = False
        else:
            print('Please upload a kit image to continue.')
            bulk_analyze_btn.disabled = True
            bulk_img_path = None
            bulk_img_bytes = None

bulk_upload.observe(on_bulk_upload_change, names='value')

# Function to analyze bulk kit image
def analyze_bulk_kit(b):
    with bulk_output:
        clear_output()
        if not bulk_img_bytes:
            print('Please upload a kit image first')
            return
        
        print('Analyzing component kit image with AI...')
        try:
            # This would call an LLM function in a real implementation
            # For now, we'll simulate with placeholder data
            kit_components = [
                {
                    'component_type': 'Resistor',
                    'value': '10K',
                    'package': '0805',
                    'quantity': 10,
                    'footprint_suggestion': 'Resistor_SMD:R_0805_2012Metric',
                    'mounting_type': 'Surface Mount'
                },
                {
                    'component_type': 'Capacitor',
                    'value': '100nF',
                    'package': '0603',
                    'quantity': 20,
                    'footprint_suggestion': 'Capacitor_SMD:C_0603_1608Metric',
                    'mounting_type': 'Surface Mount'
                }
            ]
            
            # Display detected components
            print('\n✅ Kit analysis complete!')
            print(f'\nDetected {len(kit_components)} components in kit:')
            
            # Create a table of components
            kit_df = pd.DataFrame(kit_components)
            display(kit_df)
            
            # Provide option to import all or select components
            print('\nUse the individual component workflow to add these to your inventory')
            print('Bulk import functionality coming soon!')
            
        except Exception as e:
            print(f"Error analyzing kit image: {e}")

# Connect bulk analyze button
bulk_analyze_btn.on_click(analyze_bulk_kit)